In [1]:
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

import re

import random
random.seed(2016)
pd.set_option("display.max_colwidth", 200)

# LOC = '/home/ec2-user/data/hd/unpacked/'
# df_train = pd.read_csv(LOC + 'train.csv', encoding="ISO-8859-1")
# num_train = df_train.shape[0]

In [2]:
dtrain = xgb.DMatrix("train.buffer")
dtest = xgb.DMatrix("test.buffer")
evallist  = [(dtrain,'train')]

In [21]:
loc = '%s'
#loc = '/home/ec2-user/data/hd/features/%s'
a_o = np.load(loc % 'train_data.npy')
b_o = np.load(loc % 'test_data.npy')
a_brand = np.load(loc % 'features_brand_01_train.npy')
b_brand = np.load(loc % 'features_brand_01_test.npy')
a_other = np.load(loc % 'FEATURES_1d_TRAIN.npy')
b_other = np.load(loc % 'FEATURES_1d_TEST.npy')
a_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TRAIN.npy')
b_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TEST.npy')


a_word_cooc = np.load(loc % 'WORD_COOCCUR_FEATURES_TRAIN.npy')
b_word_cooc = np.load(loc % 'WORD_COOCCUR_FEATURES_TEST.npy')
#leak:
a_word_cooc = a_word_cooc[:,[0,3]]
b_word_cooc = b_word_cooc[:,[0,3]]


aaa = pd.read_pickle(loc % 'WOQTA_TRAIN_147')
bbb = pd.read_pickle(loc % 'WOQTA_TEST_147')
aa = aaa.drop('relevance', axis=1).values[:,[0,3]]
bb = bbb.drop('relevance', axis=1).values[:,[0,3]]


a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa))
b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb))

In [ ]:
#FINAL GENERATE
aaa = pd.read_pickle(loc % 'WOQTA_ALL_TRAIN')
bbb = pd.read_pickle(loc % 'WOQTA_ALL_TEST')
aa = aaa.drop('relevance', axis=1).values[:,[0,3]]
bb = bbb.drop('relevance', axis=1).values[:,[0,3]]


a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa))
b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb))

In [19]:
for i,r in enumerate(a_word_feat):
    if np.any(np.isnan(r)):
        print i

3782
5302
8832
8999
9105
17860
17946
18072
18109
22214
22456
22751
24922
26487
31249
34447
35506
37412
39088
51361


In [ ]:
a

In [23]:
X_train, X_test, y_train, y_test = train_test_split(a, dtrain.get_label(), test_size=0.20, random_state=147) # 0.20, 147
gX_train = xgb.DMatrix(data=X_train, label=y_train)
gX_test = xgb.DMatrix(data=X_test, label=y_test)
evallist  = [(gX_train,'train'),(gX_test,'test')]

In [24]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

In [25]:
#clf = linear_model.LinearRegression(n_jobs=8)
clf = linear_model.Ridge (alpha = .6)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
y_hat = np.minimum(np.maximum(y_hat, 1.0), 3.0)
fmean_squared_error(y_hat, y_test)

0.47086068610941956

In [ ]:
# TRAIN AS SELF
param = {'max_depth':9, 
         'eta':0.1, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.3,
         'subsample':0.9,
         'nthread':8,
         'silent': True
        }
# GOOD
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.9, #7
         'subsample':0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)

In [ ]:
# 700x trees  -  cv: 0.4377 (seed 147)
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.6, #0.8 #7  0.4449
         'subsample':0.8, #0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }  
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)

In [ ]:
ggX_train = xgb.DMatrix(data=a, label=dtrain.get_label())
num_round = 500
bst = xgb.train( param, ggX_train, num_round, [(gX_test,'test')], verbose_eval=10)

idx_train = pd.read_pickle(loc % 'LABELS_TRAIN.df')
idx_test = pd.read_pickle(loc % 'LABELS_TEST.df')

ggX_test = xgb.DMatrix(data=b)
y_pred = bst.predict(ggX_test)
y_pred_bounded = np.minimum(np.maximum(y_pred, 1.0), 3.0)
idx_test['relevance'] = y_pred_bounded
idx_test.to_csv('submission_xgboost_words_0405_1446.csv')

In [ ]:
# 1500 trees 
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.8, #0.8 #7  0.4449
         'subsample':0.9, #0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }  
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)

with 0.30 and 1479
cv:  0.4493